In [1]:
#import multiple files with names as Kickstarter*.csv into pandas dataframe (data used 15 Aug 2019)

from glob import glob
import pandas as pd

# for auto-reloading extenrnal modules
%load_ext autoreload
%autoreload 2

#just replace the path parts
#glob import all files starting with Kickstarter in the folder (the folder i used is the 15 Aug 2019)

filenames = glob(r"Kickstarter_2019-08-15T03_20_03_022Z/Kickstarter*.csv")

df = [pd.read_csv(f, encoding='utf-8') for f in filenames]

data = pd.concat(df, axis = 0, ignore_index = True).reset_index()

In [2]:
from pandas.io.json import json_normalize
import json as json
import numpy as np

#data['category'].fillna(json.dumps('{}'), inplace = True)
data['location'].fillna('{}', inplace = True)
#data['creator'].fillna(json.dumps('{}'), inplace = True)

x1 = json_normalize(data['category'].apply(json.loads)).add_prefix('category_')
x3 = json_normalize(data['location'].apply(json.loads)).add_prefix('location_')
x5 = json_normalize(data['creator'].apply(lambda x: json.loads(x.replace('\n', '').replace('\r', '').replace('\\', '').replace('"', r'\"').replace(r'{\"', '{"').replace(r'\"}', '"}').replace(r':\"', ':"').replace(r'\":', '":').replace(r',\"', ',"').replace(r'\",', '",').replace(r'", ', r'\",')))).add_prefix('creator_')

data = pd.concat([data, x1, x3, x5], axis = 1)

In [3]:
from datetime import datetime
data['launched_at_utc'] = data['launched_at'].apply(datetime.utcfromtimestamp)
data['deadline_utc'] = data['deadline'].apply(datetime.utcfromtimestamp)
data['created_at_utc'] = data['created_at'].apply(datetime.utcfromtimestamp)
data['goal'] = round(data['goal'] * data['static_usd_rate'], 2)

In [4]:
#pick the relevant data and drop all irrelevant data 

df = data[['id', 'backers_count', 'fx_rate', 
'goal', 'staff_pick', 'state', 'location_id', 'creator_id',  'category_id', 'deadline_utc', 'launched_at_utc']]

#feature creation 
#feature 1: no. of projects for each creator 
df1 = df.groupby('creator_id')['id'].count()
df = pd.merge(df, pd.DataFrame(df1), how = 'left', on = 'creator_id')

#feature 2: duration of project 
df['duration'] = (df['deadline_utc'] - df['launched_at_utc']).dt.days

#feature 4: month of launch
df['launch_month'] = df['launched_at_utc'].dt.month

#change state to numerical 
df['state'] = df['state'].map({'successful': 1, 'failed': 0 })

#change staff_pick to numerical 
df['staff_pick'] = df['staff_pick'].map({True: 1, False: 0 })

#drop columns
df = df.drop(columns = ['id_x', 'launched_at_utc', 'deadline_utc', 'creator_id'])
df = df.dropna() #drop null rows

In [12]:
print(df.shape)
print(df)
print(df.iloc[0])

(192137, 10)
        backers_count   fx_rate      goal  staff_pick  state  location_id  \
0                  10  1.000000     500.0           0    0.0    2483145.0   
1                   2  1.000000  100000.0           0    0.0    2406080.0   
2                  11  1.000000    1500.0           1    0.0    2357024.0   
4                   3  1.000000    4000.0           0    0.0    2510436.0   
5                  29  1.000000    6000.0           0    0.0    2430300.0   
...               ...       ...       ...         ...    ...          ...   
207616             10  1.000000    1000.0           0    1.0   12485627.0   
207617             23  1.000000     400.0           0    1.0    2367105.0   
207618             61  1.205515     300.0           0    1.0      19344.0   
207619              8  1.114773     179.0           0    1.0     673361.0   
207620             22  0.104002    4000.0           0    1.0     901163.0   

        category_id  id_y  duration  launch_month  
0         